In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_14.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,PIT,6/14/2021,1,624428,Adam Frazier,1,N,NaN
1,PIT,6/14/2021,1,663647,Ke'Bryan Hayes,2,N,NaN
2,PIT,6/14/2021,1,668804,Bryan Reynolds,3,N,NaN
3,PIT,6/14/2021,1,607732,Jacob Stallings,4,N,NaN
4,PIT,6/14/2021,1,595943,Phillip Evans,5,N,NaN
...,...,...,...,...,...,...,...,...
241,SEA,6/14/2021,1,664238,Dylan Moore,6,N,NaN
242,SEA,6/14/2021,1,641343,Jake Bauers,7,N,NaN
243,SEA,6/14/2021,1,608596,Tom Murphy,8,N,NaN
244,SEA,6/14/2021,1,666211,Taylor Trammell,9,N,NaN


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'WAS': 'PIT',
 'PIT': 'WAS',
 'NYM': 'CHC',
 'CHC': 'NYM',
 'BOS': 'TOR',
 'TOR': 'BOS',
 'CLE': 'BAL',
 'BAL': 'CLE',
 'MIL': 'CIN',
 'CIN': 'MIL',
 'KC': 'DET',
 'DET': 'KC',
 'CHW': 'TB',
 'TB': 'CHW',
 'STL': 'MIA',
 'MIA': 'STL',
 'COL': 'SD',
 'SD': 'COL',
 'OAK': 'LAA',
 'LAA': 'OAK',
 'SF': 'ARI',
 'ARI': 'SF',
 'LAD': 'PHI',
 'PHI': 'LAD',
 'SEA': 'MIN',
 'MIN': 'SEA'}

In [12]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [13]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
    if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
            opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
            opp_pitcher.append(opp_pitch)
            keep_pitch.append(opp_pitch)
    else:
        del_indexes.append(i)
        
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-9eb6f881f0bf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-13-9eb6f881f0bf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [14]:
del_indexes

[104, 105, 106, 107, 108, 109, 110, 111, 112]

In [15]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-15-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [16]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,PIT,6/14/2021,1,624428,Adam Frazier,1,N,NaN,L,Jon Lester,...,0.263158,0.320388,0.400000,0.720388,0.136842,0.302632,12.389302,-0.048698,0.315698,98.911367
1,PIT,6/14/2021,1,663647,Ke'Bryan Hayes,2,N,NaN,L,Jon Lester,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PIT,6/14/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Jon Lester,...,0.293478,0.375000,0.510870,0.885870,0.217391,0.403226,17.904202,5.349609,0.378643,140.803722
3,PIT,6/14/2021,1,607732,Jacob Stallings,4,N,NaN,L,Jon Lester,...,0.265625,0.389610,0.546875,0.936485,0.281250,0.302326,14.800571,5.456544,0.400681,154.733712
4,PIT,6/14/2021,1,595943,Phillip Evans,5,N,NaN,L,Jon Lester,...,0.236842,0.309524,0.342105,0.651629,0.105263,0.275862,4.369694,-0.652318,0.290388,83.287519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,SEA,6/14/2021,1,641584,Jake Fraley,5,N,NaN,R,Kenta Maeda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,SEA,6/14/2021,1,664238,Dylan Moore,6,N,NaN,R,Kenta Maeda,...,0.210227,0.317073,0.420455,0.737528,0.210227,0.269231,26.515307,1.648680,0.322889,109.697295
242,SEA,6/14/2021,1,641343,Jake Bauers,7,N,NaN,R,Kenta Maeda,...,0.184211,0.295455,0.289474,0.584928,0.105263,0.222222,7.167731,-3.092486,0.265402,65.885243
243,SEA,6/14/2021,1,608596,Tom Murphy,8,N,NaN,R,Kenta Maeda,...,0.132075,0.178571,0.264151,0.442722,0.132075,0.156250,1.470206,-5.059023,0.196329,25.813338


In [17]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [19]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [20]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,PIT,6/14/2021,1,624428,Adam Frazier,1,N,NaN,L,Jon Lester,...,7.363636,7.363636,7.363636,7.363636,7.363636,7.363636,7.363636,7.363636,7.363636,7.363636
1,PIT,6/14/2021,1,663647,Ke'Bryan Hayes,2,N,NaN,L,Jon Lester,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PIT,6/14/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Jon Lester,...,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751
3,PIT,6/14/2021,1,607732,Jacob Stallings,4,N,NaN,L,Jon Lester,...,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751
4,PIT,6/14/2021,1,595943,Phillip Evans,5,N,NaN,L,Jon Lester,...,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751,6.025751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,SEA,6/14/2021,1,641584,Jake Fraley,5,N,NaN,R,Kenta Maeda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,SEA,6/14/2021,1,664238,Dylan Moore,6,N,NaN,R,Kenta Maeda,...,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216
242,SEA,6/14/2021,1,641343,Jake Bauers,7,N,NaN,R,Kenta Maeda,...,9.450000,9.450000,9.450000,9.450000,9.450000,9.450000,9.450000,9.450000,9.450000,9.450000
243,SEA,6/14/2021,1,608596,Tom Murphy,8,N,NaN,R,Kenta Maeda,...,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216,10.216216


In [21]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [22]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [23]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,PIT,6/14/2021,1,664141,JT Brubaker,SP,N,NaN,3,5,...,9.519231,1.244344,0.248869,0.995475,1.200000,5.673077,1.340580,3.125000,23.878713,17.476676
17,WAS,6/14/2021,1,452657,Jon Lester,SP,N,NaN,2,6,...,10.429185,0.906977,0.436047,0.470930,1.657143,8.652361,1.734940,4.659574,32.563848,31.511469
26,CHC,6/14/2021,1,453562,Jake Arrieta,SP,N,NaN,4,4,...,5.323944,0.666667,0.282828,0.383838,1.092896,5.408451,1.202797,3.043478,18.575007,17.668701
35,NYM,6/14/2021,1,656849,David Peterson,SP,N,NaN,3,5,...,7.834821,1.022364,0.575080,0.447284,1.080586,6.361607,1.173469,3.589109,26.614564,24.147338
45,TOR,6/14/2021,1,666201,Alek Manoah,SP,N,NaN,2,7,...,9.450000,2.240000,0.280000,1.960000,0.913043,4.200000,1.000000,7.000000,25.113054,26.125458
55,BOS,6/14/2021,1,543135,Nathan Eovaldi,SP,N,NaN,3,6,...,7.087500,1.417722,0.227848,1.189873,1.910314,9.000000,2.400000,5.026596,21.319211,17.477623
65,BAL,6/14/2021,1,665152,Dean Kremer,SP,N,NaN,4,5,...,7.258064,0.985915,0.422535,0.563380,1.523438,8.225806,1.752577,3.522727,23.027011,24.914482
75,CLE,6/14/2021,1,650496,Jean Carlos Mejia,SP,N,NaN,4,5,...,0.000000,2.000000,0.333333,1.666667,0.357143,2.000000,0.625000,5.000000,6.687896,8.433420
84,CIN,6/14/2021,1,661269,Vladimir Gutierrez,SP,N,NaN,4,4,...,6.000000,0.695652,0.173913,0.521739,0.761905,3.333333,0.705882,2.222222,17.883650,18.526446
93,MIL,6/14/2021,1,641778,Eric Lauer,SP,N,NaN,2,6,...,19.702703,1.573770,0.590164,0.983607,2.057143,11.675676,2.382353,3.956044,41.312990,25.408767


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [24]:
scores = pd.read_csv('../datasets/6_14_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210614,146x,596295,"Gomber, Austin",1.0,9,1,28.2,7000,COL,v sdg,NaN,3,2
1,20210614,116s,622072,"Wood, Alex",1.0,9,1,23.9,8500,SFO,v ari,NaN,5,2
2,20210614,166f,661269,"Gutierrez, Vladimir",1.0,9,1,21.9,6900,CIN,@ mil,NaN,10,2
3,20210614,159n,656849,"Peterson, David",1.0,9,1,21.7,7600,NYM,v chc,NaN,5,2
4,20210614,108f,543135,"Eovaldi, Nathan",1.0,0,1,20.6,8600,BOS,v tor,NaN,2,1


In [25]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [26]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [27]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [28]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [29]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [30]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [31]:
bat_final.to_csv('../merged_data/hitters_6_14.csv')
pitch_final.to_csv('../merged_data/pitchers_6_14.csv')

In [32]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,PIT,6/14/2021,1,624428,Adam Frazier,1,N,NaN,L,Jon Lester,...,1.0,1,4,2.0,4000,PIT,@ was,NaN,2,3
2,PIT,6/14/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Jon Lester,...,1.0,3,7,5.0,3300,PIT,@ was,NaN,2,3
3,PIT,6/14/2021,1,607732,Jacob Stallings,4,N,NaN,L,Jon Lester,...,1.0,4,2,7.0,4000,PIT,@ was,NaN,2,3
4,PIT,6/14/2021,1,595943,Phillip Evans,5,N,NaN,L,Jon Lester,...,1.0,5,37,2.0,2500,PIT,@ was,NaN,2,3
5,PIT,6/14/2021,1,570481,Erik Gonzalez,6,N,NaN,L,Jon Lester,...,1.0,7,35,5.0,2200,PIT,@ was,NaN,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,SEA,6/14/2021,1,572122,Kyle Seager,3,N,NaN,R,Kenta Maeda,...,1.0,4,5,5.0,3400,SEA,v min,NaN,4,3
172,SEA,6/14/2021,1,664034,Ty France,4,N,NaN,R,Kenta Maeda,...,1.0,3,3,11.0,3000,SEA,v min,NaN,4,3
174,SEA,6/14/2021,1,664238,Dylan Moore,6,N,NaN,R,Kenta Maeda,...,1.0,5,47,10.0,2600,SEA,v min,NaN,4,3
175,SEA,6/14/2021,1,641343,Jake Bauers,7,N,NaN,R,Kenta Maeda,...,1.0,6,3,14.0,2700,SEA,v min,NaN,4,3


In [33]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,PIT,6/14/2021,1,664141,JT Brubaker,SP,N,NaN,3,5,...,1.0,9,1,5.65,7300,PIT,@ was,NaN,2,3
1,WAS,6/14/2021,1,452657,Jon Lester,SP,N,NaN,2,6,...,1.0,9,1,7.80,6700,WAS,v pit,NaN,3,2
2,CHC,6/14/2021,1,453562,Jake Arrieta,SP,N,NaN,4,4,...,1.0,9,1,4.45,6900,CHC,@ nym,NaN,2,5
3,NYM,6/14/2021,1,656849,David Peterson,SP,N,NaN,3,5,...,1.0,9,1,21.70,7600,NYM,v chc,NaN,5,2
4,TOR,6/14/2021,1,666201,Alek Manoah,SP,N,NaN,2,7,...,1.0,0,1,18.50,8200,TOR,@ bos,NaN,1,2
5,BOS,6/14/2021,1,543135,Nathan Eovaldi,SP,N,NaN,3,6,...,1.0,0,1,20.60,8600,BOS,v tor,NaN,2,1
6,BAL,6/14/2021,1,665152,Dean Kremer,SP,N,NaN,4,5,...,1.0,0,1,7.00,6200,BAL,@ cle,NaN,3,4
7,CIN,6/14/2021,1,661269,Vladimir Gutierrez,SP,N,NaN,4,4,...,1.0,9,1,21.90,6900,CIN,@ mil,NaN,10,2
8,MIL,6/14/2021,1,641778,Eric Lauer,SP,N,NaN,2,6,...,1.0,9,1,3.85,7900,MIL,v cin,NaN,2,10
9,KC,6/14/2021,1,641745,Brad Keller,SP,N,NaN,5,4,...,1.0,0,1,3.25,6400,KAN,v det,NaN,3,10
